In [191]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import warnings
from tqdm.notebook import tqdm
import random
from scipy.stats import norm
warnings.simplefilter(action='ignore', category=FutureWarning or RuntimeWarning)
DATA_PATH = "../model_code/"

### CB 모델의 성능을 검증한다.

In [21]:
# 메모리상에 여유를 위해 pickling 하는 함수
def pickling(arg_object, arg_file_name):
    with open(f'{arg_file_name}.pkl','wb') as pickle_file:
        pickle.dump(arg_object, pickle_file)       
    print(f"{arg_file_name}.pkl로 pickling 완료")

In [22]:
# 저장한 pkl 파일을 불러오는 함수
def test_pkl(name):
    test = None
    with open(f'{name}.pkl','rb') as pickle_file:
        test = pickle.load(pickle_file)
    return test

### 각 카테고리별로 pearson 유사도를 계산해서 해당 유저가 가장 많이 본 상품과 비슷한 상품 10개를 추천한다

In [23]:
# 각 카테고리별로 pearson table을 만들어서 저장한다.
# CB를 위한 Persontable을 category에 따라 pcikling
# pearson_table을 만들기 위한 데이터 따로 저장
df = pd.read_parquet(DATA_PATH + "view_data.parquet.gzip", columns = ["category_code"])
df = df.fillna("missing")
df["category_code"] = df["category_code"].apply(lambda x : x.split("."))
df["category_code_0"] = df["category_code"].apply(lambda x : x[0])
category_code_list = list(df["category_code_0"].unique())
category_code_list.remove("missing")
del df

def df_by_category(target_category = "all"):
    """
    매개변수
    target_category : 1차 카테고리 코드를 지정해준다. all이면 모든 카테고리를 가져온다.
    10개 이하의 product를 view한 user만 가져온다. 
    """
    df = pd.read_parquet(DATA_PATH + "view_data.parquet.gzip", columns = ["user_id",	"event_type",	"product_id",	"category_id",	"category_code",	"brand",	"price"])
    df = df.fillna("missing")
    df["event_type"] = [1] * df.shape[0]
    df.rename(columns = {'event_type':'event_type_view'},inplace=True)
    df["category_code"] = df["category_code"].apply(lambda x : x.split("."))
    df["category_code_0"] = df["category_code"].apply(lambda x : x[0])
    
    # 카테고리에 적합한 df불러오기
    if target_category != "all" :
        df = df[df["category_code_0"] == target_category]
    df = df[df['brand'] != 'missing']
    df = df[['user_id','event_type_view', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'category_code_0']]
    
    # 기존 frame에 innerjoin
    df_user_view = df.groupby(['user_id']).agg({'event_type_view' : 'sum'}).reset_index()
    df_pick = pd.merge(left = df , right = df_user_view[['user_id']], how = "inner", on = "user_id")
    del df_user_view , df
    
    # view한 product가 2개이상 10개 이하인 고객만 가져온다.
    df_view_prod = df_pick.groupby(['user_id'])['product_id'].agg({'unique'})
    df_view_prod['view_prod'] =df_view_prod['unique'].apply(lambda x: len(x))
    df_view_prod = df_view_prod[(df_view_prod['view_prod'] >= 2) & (df_view_prod['view_prod'] <= 10)]
    df_view_prod = df_view_prod.reset_index()
    df_pick = pd.merge(left = df_pick , right = df_view_prod[['user_id']], how = "inner", on = "user_id")
    del df_view_prod
    
    # prod_id별 mean_price 삽입
    df_prod_id = df_pick.groupby(['product_id']).agg({'price' : 'mean'}).reset_index()
    df_prod_id.rename(columns = {'price':'mean_price'},inplace=True)
    df_pick = pd.merge(left = df_pick , right = df_prod_id, how = "inner", on = "product_id")
    del df_prod_id
    
    # minMaxScale
    scaler = MinMaxScaler()
    scaler.fit(df_pick[['mean_price']])

    mMscaled_data = scaler.transform(df_pick[['mean_price']])
    mMscaled_data = pd.DataFrame(mMscaled_data, columns=['mMprice'])
    df_pick = pd.concat([df_pick, mMscaled_data], axis= 1)
    df_pick["category_code"] = df_pick["category_code"].apply(lambda x : str(x))
    
    del mMscaled_data, scaler
    
    return df_pick
    
def to_matrix(df):
    
    # Tfidf 사전 df생성
    df = df[['product_id', 'category_code', 'brand','mean_price', 'mMprice']]
    df = df.groupby(['product_id']).agg({'category_code' : 'unique', 'brand' : 'unique',
                                        'mMprice' : 'unique', 'mean_price' : 'unique'}).reset_index()
    df['category_code'] = df['category_code'].apply(lambda x: x[0])
    df['category_code'] =df['category_code'].apply(lambda row: eval(''.join(row)))
    df['category_code'] =df['category_code'].apply(lambda row: '.'.join(row))
    df['brand'] = df['brand'].apply(lambda x: x[0])
    df['mean_price'] = df['mean_price'].apply(lambda x: x[0])
    df['mMprice'] = df['mMprice'].apply(lambda x: x[0])

    cols = ['category_code', 'brand']
    df['category_code+brand'] =df[cols].apply(lambda row: '.'.join(row.values.astype(str)), axis=1)
    df = df[['product_id', 'category_code+brand', 'mean_price', 'mMprice']]
    df['category_code+brand'] = df['category_code+brand'].str.replace('.',' ')
    
    vect = CountVectorizer()
    docs = df['category_code+brand'].values
    countvect = vect.fit_transform(docs)

    # 벡터 데이터프레임화
    countvect_df =pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
    countvect_df.index = df['product_id'].values
    
    df_tfidf_con = df[['mMprice']]
    df_tfidf_con = df_tfidf_con.set_index(countvect_df.index)
    countvect_df = pd.concat([countvect_df,df_tfidf_con], axis = 1)
    del df_tfidf_con
    
    # pearson matrix 생성
    pearson_sim = countvect_df.T.corr()
    
    del countvect_df, vect, docs

    return pearson_sim        
for category_code in category_code_list:
    df = df_by_category(target_category = category_code)
    df = to_matrix(df)
    pickling(df, f"{category_code}_pearson_table")

C:\Users\havea\AppData\Local\Temp/ipykernel_14980/1209434548.py:79: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['category_code+brand'] = df['category_code+brand'].str.replace('.',' ')


appliances_pearson_table.pkl로 pickling 완료


C:\Users\havea\AppData\Local\Temp/ipykernel_14980/1209434548.py:79: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['category_code+brand'] = df['category_code+brand'].str.replace('.',' ')


furniture_pearson_table.pkl로 pickling 완료


C:\Users\havea\AppData\Local\Temp/ipykernel_14980/1209434548.py:79: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['category_code+brand'] = df['category_code+brand'].str.replace('.',' ')


...

C:\Users\havea\AppData\Local\Temp/ipykernel_14980/1209434548.py:79: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['category_code+brand'] = df['category_code+brand'].str.replace('.',' ')


medicine_pearson_table.pkl로 pickling 완료


C:\Users\havea\AppData\Local\Temp/ipykernel_14980/1209434548.py:79: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['category_code+brand'] = df['category_code+brand'].str.replace('.',' ')


stationery_pearson_table.pkl로 pickling 완료
country_yard_pearson_table.pkl로 pickling 완료


C:\Users\havea\AppData\Local\Temp/ipykernel_14980/1209434548.py:79: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['category_code+brand'] = df['category_code+brand'].str.replace('.',' ')


In [24]:
del df

In [25]:
# product_id 입력받으면 해당 제품의 1차 카테고리를 반환하는 dict 생성
df = pd.read_parquet(DATA_PATH + "view_data.parquet.gzip", columns = ["product_id", "category_code"])
df = df.fillna("missing")
df = df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=True)
df["category_code"] = df["category_code"].apply(lambda x : x.split(".")[0])
product_id_to_category_code = {product_id : category_code for product_id, category_code in list(zip(df["product_id"], df["category_code"]))}
pickling(product_id_to_category_code, "product_id_to_category_code")
del product_id_to_category_code, df

product_id_to_category_code.pkl로 pickling 완료


In [83]:
# 10개 이하, 2개 이상의 product 를 view한 user_id를 받으면 가장 view가 많은 상품을 반환하는 dict 저장 (단 category_code나 brand가 결측값인 로그는 제거한다)
# user_to_most_viewed_product_id
df = pd.read_parquet(DATA_PATH + "view_data.parquet.gzip", columns = ["user_id", "product_id", "category_code", "brand"])
df = df.dropna().reset_index(drop =True)
df = df[["user_id", "product_id"]]
df = df.groupby("user_id").nunique()
df = df[(df["product_id"] >= 2) & (df["product_id"] <= 10)]
lower_user_list = df.index
df = pd.read_parquet(DATA_PATH + "view_data.parquet.gzip", columns = ["user_id", "product_id", "event_type", "category_code", "brand"])
df = df[df["user_id"].isin(lower_user_list)]  
df = df.dropna().reset_index(drop =True)
df = df[["user_id", "product_id", "event_type"]]
df = df.groupby(["user_id", "product_id"]).count().reset_index()

# 데이터를 csr_matrix로 만드는 과정입니다.
user_unique = df['user_id'].unique()
product_unique = df['product_id'].unique()
cb_user_to_index = {user:index for index, user in enumerate(user_unique)}
cb_index_to_user = {index:user for index, user in enumerate(user_unique)}
cb_product_to_index = {product:index for index, product in enumerate(product_unique)}
cb_index_to_product = {index:product for index, product in enumerate(product_unique)}

# user와 product 의 index와의 변환을 위한 dict pickling
df['user_id'] = df['user_id'].map(cb_user_to_index.get)
df['product_id'] = df['product_id'].map(cb_product_to_index.get)
num_user = df['user_id'].nunique()
num_product = df['product_id'].nunique()
lower_user_item_matrix = csr_matrix((df.event_type, (df.user_id, df.product_id)), shape= (num_user, num_product))

del df, num_product, num_user, user_unique, product_unique

### 성능 평가를 위해 한 유저당 임의의 상품 하나의 view기록을 0으로 만듬

In [84]:
# 각 유저마다 랜덤하게 하나씩 0으로 가리는 작업입니다.
samples = []

for user_idx in tqdm(range(lower_user_item_matrix.shape[0])) :
    samples.append((user_idx, random.sample(lower_user_item_matrix[user_idx].nonzero()[1].tolist(), 1)[0]))
    
training_set = lower_user_item_matrix.copy()
test_set = lower_user_item_matrix.copy()

user_inds = [index[0] for index in samples]
item_inds = [index[1] for index in samples]

training_set[user_inds, item_inds] = 0
training_set.eliminate_zeros()

del lower_user_item_matrix

  0%|          | 0/1177302 [00:00<?, ?it/s]

In [85]:
# 유저별로 가장 view 수가 큰 product_id를 가지는 list
input_data = list(np.array(np.argmax(training_set, axis=1)).reshape(-1))
input_data = list(map(cb_index_to_product.get, input_data))

# 가려진 product_id를 가지는 list
label = list(map(cb_index_to_product.get, item_inds))

In [146]:
# 표본으로 정확도 검증 하기
df = pd.read_parquet(DATA_PATH + "view_data.parquet.gzip", columns = ["product_id", "event_type"])
df = df.groupby("product_id").count()
popular_product_id_list = list(df.sort_values("event_type", ascending=False).index[:10])
del df
product_id_to_category_code = test_pkl("product_id_to_category_code")
answer_store_by_model = []
answer_store_by_pop = []
sample_size = 10000
for user_index in tqdm(np.random.randint(training_set.shape[1], size=sample_size)):
    input_product_id = input_data[user_index]
    input_category_code = product_id_to_category_code[input_product_id]
    pearson_table = test_pkl(f"{input_category_code}_pearson_table")
    viewed_product_index_list = list(np.where(training_set[user_index].toarray()[0] != 0)[0])
    viewed_product_id_list = list(map(cb_index_to_product.get, viewed_product_index_list ))
    pearson_table = pearson_table[~pearson_table.index.isin(viewed_product_id_list)]
    answer_by_model = label[user_index] in list(pearson_table[input_product_id].sort_values(ascending=False).index[:10])
    answer_store_by_model.append(answer_by_model)
    answer_by_pop = label[user_index] in popular_product_id_list
    answer_store_by_pop.append(answer_by_pop)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [190]:
# 귀무가설 : cb 모델을 사용하는것의 정확도와 가장조회수가 높은 것을 추천하는것의 정확도보다 작거나 같다.
# 대립가설 : cb 모델을 사용하는것의 정확도가 가장조회수가 높은 것을 추천하는것의 정확도보다 크다.

model = sum(answer_store_by_model)/len(answer_store_by_model)
pop = sum(answer_store_by_pop)/len(answer_store_by_model)
pool = (sample_size * (model + pop)) / (sample_size * 2)
Z = (model - pop) / np.sqrt(pool * (1 - pool) * (1/sample_size + 1/sample_size))
print(f"검정통계량 값 = {Z}")
print(f"유의확률 = {1 - norm.cdf(Z)}")

검정통계량 값 = 3.5819902492699147
유의확률 = 0.00017049326141016508


유의수준 0.01 하에서 귀무가설을 기각한다.  
cb 모델을 사용하는것의 정확도가 가장조회수가 높은 것을 추천하는것의 정확도보다 크다.